# QA봇
https://www.sbert.net/docs/hugging_face.html#sharing-your-models

In [ ]:
!pip install sentence_transformers

In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
train_data.drop(columns='label',inplace=True)
train_data

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.
...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.


In [ ]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [ ]:
train_data.head(3)

,Q,A,embedding
0,12시 땡!,하루가 또 가네요.,"[-0.041483987, 0.12033884, -0.024771892, -0.07..."
1,1지망 학교 떨어졌어,위로해 드립니다.,"[-0.058250703, 0.11278771, -0.00276401, -0.010..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,"[-0.029141849, 0.066955626, -0.027986377, -0.0..."


In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def return_answer(question):
  embedding = model.encode(question)
  train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
  return train_data.loc[train_data['score'].idxmax()]['A']

In [ ]:
return_answer('누구?')

'저는 마음을 이어주는 위로봇입니다.'

# 외부 데이터 → 결국 사용하지 못했지만 누군가는 이어주길..★
AI hub 일반상식

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # json 파일을 데이터프레임으로 변경
# import json
# import pandas as pd

# df = pd.read_json('/content/drive/MyDrive/3차 프로젝트_Early Bird팀/최지원/ko_wiki_v1_squad.json')
# df

,creator,version,data
0,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': '다테 기미코가...
1,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': 'ave;new...
2,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': '사카이 다다요...
3,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': '일반성면의 면...
4,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': '금나라를 세운...
...,...,...,...
68533,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': '네 가지 베다...
68534,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': '보안사 민간인...
68535,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': 'FM 음악도시...
68536,MINDs Lab.,1,{'paragraphs': [{'qas': [{'question': '파푸아뉴기니는...


In [ ]:
# def flatten_nested_json_df(df):

#     df = df.reset_index()

#     print(f"original shape: {df.shape}")
#     print(f"original columns: {df.columns}")


#     # search for columns to explode/flatten
#     s = (df.applymap(type) == list).all()
#     list_columns = s[s].index.tolist()

#     s = (df.applymap(type) == dict).all()
#     dict_columns = s[s].index.tolist()

#     print(f"lists: {list_columns}, dicts: {dict_columns}")
#     while len(list_columns) > 0 or len(dict_columns) > 0:
#         new_columns = []

#         for col in dict_columns:
#             print(f"flattening: {col}")
#             # explode dictionaries horizontally, adding new columns
#             horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
#             horiz_exploded.index = df.index
#             df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
#             new_columns.extend(horiz_exploded.columns) # inplace

#         for col in list_columns:
#             print(f"exploding: {col}")
#             # explode lists vertically, adding new columns
#             df = df.drop(columns=[col]).join(df[col].explode().to_frame())
#             new_columns.append(col)

#         # check if there are still dict o list fields to flatten
#         s = (df[new_columns].applymap(type) == list).all()
#         list_columns = s[s].index.tolist()

#         s = (df[new_columns].applymap(type) == dict).all()
#         dict_columns = s[s].index.tolist()

#         print(f"lists: {list_columns}, dicts: {dict_columns}")

#     print(f"final shape: {df.shape}")
#     print(f"final columns: {df.columns}")
#     return df

In [ ]:
# df = flatten_nested_json_df(df)

original shape: (68538, 4)
original columns: Index(['index', 'creator', 'version', 'data'], dtype='object')
lists: [], dicts: ['data']
flattening: data
lists: ['data.paragraphs'], dicts: []
exploding: data.paragraphs
lists: [], dicts: ['data.paragraphs']
flattening: data.paragraphs
lists: ['data.paragraphs.qas'], dicts: []
exploding: data.paragraphs.qas
lists: [], dicts: ['data.paragraphs.qas']
flattening: data.paragraphs.qas
lists: ['data.paragraphs.qas.answers'], dicts: []
exploding: data.paragraphs.qas.answers
lists: [], dicts: ['data.paragraphs.qas.answers']
flattening: data.paragraphs.qas.answers
lists: [], dicts: []
final shape: (278198, 9)
final columns: Index(['index', 'creator', 'version', 'data.title', 'data.paragraphs.context',
       'data.paragraphs.qas.question', 'data.paragraphs.qas.id',
       'data.paragraphs.qas.answers.answer_start',
       'data.paragraphs.qas.answers.text'],
      dtype='object')


In [ ]:
# df.head()

,index,creator,version,data.title,data.paragraphs.context,data.paragraphs.qas.question,data.paragraphs.qas.id,data.paragraphs.qas.answers.answer_start,data.paragraphs.qas.answers.text
0,0,MINDs Lab.,1,다테_기미코,"재팬 오픈에서 4회 우승하였으며, 통산 단식 200승 이상을 거두었다. 1994년 ...",다테 기미코가 최초로 은퇴 선언을 한게 언제지,9_f2_wiki_2822-1,260,1996년 9월 24일
1,1,MINDs Lab.,1,Ave;new,"ave;new(아베;뉴, アベニュー)는 도쿄 치요다구에 본 거처를 둔 일본의 음악 ...",ave;new 본거지 어디야,9_f2_wiki_3091-1,22,도쿄 치요다구
2,2,MINDs Lab.,1,사카이_다다요시_(1714년),"사카이 다다요시(일본어: 酒井忠休, 1714년 9월 24일 ~ 1787년 6월 3일...",사카이 다다요시의 아버지가 누구지,9_f7_wiki_3128-1,182,사카이 나오타카(酒井直隆)
3,3,MINDs Lab.,1,일반성면,"일반성면은 동부 5개 면의 교통, 문화, 교육, 상업의 중심지로서 일찍부터 상업이 ...",일반성면의 면적이 얼마야,9_f9_wiki_2703-1,84,19.41 km²
4,4,MINDs Lab.,1,금나라,"지방은 전국을 19개 로(路)로 나누고, 그 아래에 부(府)나 주(州)를 두고, 다...",금나라를 세운 사람이 누구야,8_C59_wiki_1533-1,247,태조 아구다


In [ ]:
# df.columns

Index(['index', 'creator', 'version', 'data.title', 'data.paragraphs.context',
       'data.paragraphs.qas.question', 'data.paragraphs.qas.id',
       'data.paragraphs.qas.answers.answer_start',
       'data.paragraphs.qas.answers.text'],
      dtype='object')

In [ ]:
# df.drop(columns=['index', 'creator', 'version', 'data.title',
#        'data.paragraphs.context','data.paragraphs.qas.id','data.paragraphs.qas.answers.answer_start'], inplace=True)

In [ ]:
# df.columns

Index(['data.paragraphs.qas.question', 'data.paragraphs.qas.answers.text'], dtype='object')

In [ ]:
# df.head(2)

,data.paragraphs.qas.question,data.paragraphs.qas.answers.text
0,다테 기미코가 최초로 은퇴 선언을 한게 언제지,1996년 9월 24일
1,ave;new 본거지 어디야,도쿄 치요다구


In [ ]:
# df.rename(columns={'data.paragraphs.qas.question':'Q', 'data.paragraphs.qas.answers.text':'A'}, inplace=True)

In [ ]:
# df

,Q,A
0,다테 기미코가 최초로 은퇴 선언을 한게 언제지,1996년 9월 24일
1,ave;new 본거지 어디야,도쿄 치요다구
2,사카이 다다요시의 아버지가 누구지,사카이 나오타카(酒井直隆)
3,일반성면의 면적이 얼마야,19.41 km²
4,금나라를 세운 사람이 누구야,태조 아구다
...,...,...
68535,FM 음악도시의 1대 진행자는,2011년 6월 1일
68535,FM 음악도시가 2006년 폐지되고 언제 다시 방송했지,신해철
68535,FM 음악도시가 2006년 폐지되고 언제 다시 방송했지,2011년 6월 1일
68536,파푸아뉴기니는 몇 개의 주가 있어,21개


In [ ]:
# # 위아래로 합치기
# data1 = pd.concat([train_data, df])
# data1.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [ ]:
# data1

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.
...,...,...
68535,FM 음악도시의 1대 진행자는,2011년 6월 1일
68535,FM 음악도시가 2006년 폐지되고 언제 다시 방송했지,신해철
68535,FM 음악도시가 2006년 폐지되고 언제 다시 방송했지,2011년 6월 1일
68536,파푸아뉴기니는 몇 개의 주가 있어,21개


In [ ]:
# data1['embedding'] = data1.apply(lambda row: model.encode(row.Q), axis = 1)

In [ ]:
## 피클로 저장해야 embedding값에 컴마가 찍힙니다.
# data1.to_pickle('chatbot.pkl')

In [ ]:
# data1

,Q,A,embedding,score
0,12시 땡!,하루가 또 가네요.,"[-0.041483987, 0.12033884, -0.024771892, -0.07...",0.513968
1,1지망 학교 떨어졌어,위로해 드립니다.,"[-0.058250703, 0.11278771, -0.00276401, -0.010...",0.629759
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,"[-0.029141849, 0.066955626, -0.027986377, -0.0...",0.586851
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,"[-0.03855583, 0.08400126, -0.017746707, -0.040...",0.567403
4,PPL 심하네,눈살이 찌푸려지죠.,"[-0.06725987, 0.14640209, -0.019904844, -0.043...",0.474076
...,...,...,...,...
68535,FM 음악도시의 1대 진행자는,2011년 6월 1일,"[-0.016387612, 0.1271683, -0.014800723, -0.070...",0.505623
68535,FM 음악도시가 2006년 폐지되고 언제 다시 방송했지,신해철,"[-0.047776967, 0.07556521, -0.023517696, -0.04...",0.511116
68535,FM 음악도시가 2006년 폐지되고 언제 다시 방송했지,2011년 6월 1일,"[-0.047776967, 0.07556521, -0.023517696, -0.04...",0.511116
68536,파푸아뉴기니는 몇 개의 주가 있어,21개,"[-0.04189622, 0.03067604, -0.010429633, -0.047...",0.608758


In [ ]:
# df = pd.read_pickle('chatbot.pkl')

In [ ]:
# df

,Q,A,embedding,score
0,12시 땡!,하루가 또 가네요.,"[-0.041483987, 0.12033884, -0.024771892, -0.07...",0.513968
1,1지망 학교 떨어졌어,위로해 드립니다.,"[-0.058250703, 0.11278771, -0.00276401, -0.010...",0.629759
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,"[-0.029141849, 0.066955626, -0.027986377, -0.0...",0.586851
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,"[-0.03855583, 0.08400126, -0.017746707, -0.040...",0.567403
4,PPL 심하네,눈살이 찌푸려지죠.,"[-0.06725987, 0.14640209, -0.019904844, -0.043...",0.474076
...,...,...,...,...
68535,FM 음악도시의 1대 진행자는,2011년 6월 1일,"[-0.016387612, 0.1271683, -0.014800723, -0.070...",0.505623
68535,FM 음악도시가 2006년 폐지되고 언제 다시 방송했지,신해철,"[-0.047776967, 0.07556521, -0.023517696, -0.04...",0.511116
68535,FM 음악도시가 2006년 폐지되고 언제 다시 방송했지,2011년 6월 1일,"[-0.047776967, 0.07556521, -0.023517696, -0.04...",0.511116
68536,파푸아뉴기니는 몇 개의 주가 있어,21개,"[-0.04189622, 0.03067604, -0.010429633, -0.047...",0.608758


In [ ]:
# def cos_sim(A, B):
#   return dot(A, B)/(norm(A)*norm(B))

In [ ]:
# def return_answer(question):
#   embedding = model.encode(question)
#   data1['score'] = data1.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
#   return data1.loc[data1['score'].idxmax()]['A']

In [ ]:
# return_answer('배고파')

2000    얼른 맛난 음식 드세요.
2000            하랄 1세
Name: A, dtype: object

스코어가 외부데이터랑 내부데이터가 같아서 동시 출력.. 결국 실패..